<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/hannahklingberg-lab2/Lab2/hannahklingberg-lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Hanna Klingberg**

# **Abstract**



```
# This is formatted as code
```

Short summary of the lab report. State the objectives, methods used, main results and conlusions. 

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [41]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [42]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import math


#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

This lab explores methods for matrix factorization and multiplication. By factorizing a matrix into other matrices of types that are easier to work with, a number of matrix calculations become easier to do. In this lab, QR-factorization is done and used to compute a directt solution of a linear equation system Ax=b. A method for multiplying with sparse matrices is also calculated. 



# **Method**

The functions, how they work and how they are tested is described in their own section. 

# **Sparse matrix-vector product**
A sparse matrix is a matrix A $\in R^{nxn}$ where most of its components are zero, more specifically if $\mathcal{O}(n)$ of its components are non-zero. A sparse matrix can be represented in more space-efficient ways, such as the Compressed Row Storage format (CRS). This format has three components:

  $\circ \textit{val}$, array containing the nonzero values of the matrix \\
  $\circ \textit{col_idx}$, array containing the column indices of the items in $\textit{val}$ \\
  $\circ \textit{row_ptr}$, array indicating the start points of each row and ends with the number of non-zero components + 1

(Reference: Chapter 5, Course Book)

The function takes a vector x, and the CRS representation of matrix A, given with the arrays val, col_idx and row_ptr. The algorithm used is the one presented in Algorithm 5.9 in the course book, but adapted to the way Python handles indices. 

The tests compare the computed sparse matrix-vector product to the actual inner product of the matrix and vector (calculated by hand).  



In [43]:
#sparse matrix-vector product
#implemented from pseudo code from Algorithm 5.9 (course book chapter 5.8)
def sparse_mat_vec_prod(x, val, col_idx, row_ptr):
  b = np.zeros(len(x))
  for i in range(len(x)):
    for j in range(row_ptr[i], row_ptr[i+1]):
      b[i] += val[j-1]*x[col_idx[j-1]-1]
  return b


In [44]:
def test_mat_vec_prod():
  passed = False
  val1 = np.array([3,2,2,2,1,1,3,2,1,2,3])
  col_idx1 = np.array([1,2,4,2,3,3,3,4,5,5,6])
  row_ptr1 = np.array([1,4,6,7,9,10,12])
  x1 = np.array([1,2,3,0,1,2])
  result1 = np.array([7,7,3,9,1,8]) #result calculated by hand
  test1 = sparse_mat_vec_prod(x1,val1,col_idx1,row_ptr1)
  if np.array_equal(test1,result1):
    passed = True
  else:
    return False
  return passed 
  


# **QR factorization**

QR factorization is the factorization of a matrix $A \in R^{nxn} $ into two matrices Q and R, such that A = QR where Q is an orthogonal matrix and R is an upper triangular matrix. Doing this can facilitate solving systems of linear equations of the form Ax=b, since finding the inverse to Q and R is easier than finding the inverse to A. 

The algorithm inmplemented in this function is the modified Gram-Schmidt iteration, where A is factorized into Q and R by multiplicating by an upper triangular matrix $R_k$ from the right side such that

Q = $AR_1R_2...R_n$
and then 
$R^{-1} = R_1R_2...R_n$, $(R^{-1})^{-1}$, which leads to A = QR
(Reference: Chapter 5.3 in Course Book)
The function is implemented after the pseudo-code given in Algorithm 5.3. It takes a matrix A as input. Q and R are initiated as zero matrices with the same shape as A, and as we iterate through A they are filled with the correct values. Q and R are returned. 

The tests assert that R is upper triangular and that the Frobenius norms of $Q^TQ-I$ and $QR-A$ are 0. The former asserts that Q is an orthogonal matrix, which gives that $Q^TQ = I$ and thus $||Q^TQ-I|| = 0$. The latter asserts that the QR factorization of A is correct. If it is, then $||QR-A||$ is 0.  

In [45]:
#QR factorization
def QR_factorization(A):
  Q = np.zeros((A.shape))
  R = np.zeros((A.shape))
  for j in range(len(A)):
    v = np.array([x[j] for x in A])
    for i in range(0,j):
      Q_col = np.array([y[i] for y in Q])
      R[i][j] = np.dot(Q_col, v)
      v = v - R[i,j]*Q_col
    R[j][j] = np.linalg.norm(v)
    Q[:,j] = v/R[j][j]
  return Q, R

In [46]:
def test_QR():
  passed = False
  Atest = np.array([[2,-1],[-1,2]])
  Qtest1, Rtest1 = QR_factorization(Atest)
  Qresult =(1/math.sqrt(5)) * np.array([[2,1], [-1,2]])
  Rresult = (1/math.sqrt(5)) * np.array([[5,-4],[0,3]])
  #is R upper triangular?
  triangular = False
  for i in range(1,len(Rtest1)):
    for j in range(i):
      if Rtest1[i][j] == 0:
        triangular = True
      else:
        triangular = False
        break
    #Frobenius Norm
    #QTQ - I
  QTQ = np.matmul(np.transpose(Qtest1), Qtest1)
  QTQI = QTQ - np.identity(len(QTQ))
  QR = np.matmul(Qtest1, Rtest1)
  #print(QR)
  QRA = QR - Atest
  frobeniusQ = 0
  frobeniusA = 0
  #print(QTQI)
  for i in range(len(QTQI)):
    for j in range(len(QTQI)):
      frobeniusQ += (QTQI[i][j])**2
      frobeniusA += (QRA[i][j])**2
  frobeniusQ = math.sqrt(frobeniusQ)
  frobeniusA = math.sqrt(frobeniusA)
  passed = np.isclose(frobeniusQ, 0, 10**-10)
  if not passed:
    return False
  passed = np.isclose(frobeniusA, 0, 10**-10)
  if not passed:
    return False
  if not triangular:
    return False
  return passed

test_QR()

True

# **Direct Solver Ax=b**
For solving the matrix equation $Ax=b$, where x is unknown, one could express the solution as $x = A^{-1}b$. But the inverse of A can be hard to calculate, which is why it might be simpler to factorize $A=QR$ as done above, because the inverses of an orthogonal matrix and upper triangular matrix is easier to construct. 

The inverse of Q is its transpose, $Q^{-1} = Q^T$, but the inverse of R has to be calculated using the backwards substitution algorithm, given as pseudocode in Algorithm 5.1. This algorithm takes an upper triangular matrix U and a vector b as input, such that Ux = b, and returns the solution vector x. 

For constructing the solver for Ax = b, I first factorize A = QR.
Then since:
$QRx = b \\ Rx = Q^{-1}b $

We can use $Q^{-1}b$ as the new b in backwards substitution, and solve for x.  

The code is implemented from the pseudocode given in algorithm 5.2, and the theory is from chapter 5.2 in the course book. 

The tests are done by comparing the residual from both $||Ax-b||$ and $||x-x_2||$, where $x_2$ is the manufactured solution and $x$ is the solution calculated by the algorithm. We assert that they are equal up to $10^{-10}$ decimal point. 


In [47]:
def backwards_substitution(U,b):
  n = b.size
  x = np.zeros(n)
  x[n-1] = b[n-1] / U[n-1][n-1]
  for i in range(n-2, -1, -1):
    sum = 0
    for j in range(i+1, n):
      sum += U[i][j]*x[j]
    x[i] = (b[i]-sum)/U[i][i]
  return x 

In [48]:
#Direct solver
def direct_solver(A,b):
  Q1, R1 = QR_factorization(A)
  x = backwards_substitution(R1, np.dot(np.transpose(Q1), b))
  return x


In [49]:
def test_direct_solver():
  passed = False
  A = np.array([[1,2],[3,4]])
  x = np.array([1,2])
  b = np.dot(A,x)
  x2 = direct_solver(A,b)
  residual1 = np.linalg.norm(np.dot(A,x2)-b) # ||Ax-b||
  residual2 = np.linalg.norm(x-x2) # ||x-y||
  passed = np.isclose(residual1, 0, 10**-10) #are residual close up to 10^-10 decimal point?
  if not passed:
    return False
  passed = np.isclose(residual2, 0, 10**-10)
  return passed

test_direct_solver()

True

# **Results**

The tests for each function is described in each function definition. If any of the described test criteria are not fulfilled, the test function will return False. All functions passed their tests. 

In [50]:
def test_functions():
  passed = 0
  sparse = test_mat_vec_prod()
  if sparse:
    passed +=1
    print("Sparse matrix-vector product passed the test")
  QR = test_QR()
  if QR: 
    passed +=1
    print("QR factorization passed the test")
  solver = test_direct_solver()
  if solver:
    passed +=1
    print("Direct solver passed the test")
  print("%d of 3 functions passed their tests" %(passed))

test_functions()

Sparse matrix-vector product passed the test
QR factorization passed the test
Direct solver passed the test
3 of 3 functions passed their tests


# **Discussion**

The functions all passed their tests. The tests are done with small matrices of 2x2 and 3x3 sizes. If larger matrices were to be used, the sparse-matrix vector product could be used to facilitate some of the calculations instead of using np.dot and similar functions. Some operations such as inner product and norm are calculated using the numpy library, since they are not central to the lab. I have not included tests that verify that the input of the functions is correct, since it was not specified to do so in the problem description. 